## LSTMAE

In [1]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, StratifiedKFold, KFold
from sklearn import linear_model, metrics, preprocessing
from math import sqrt, floor
from datetime import datetime, timedelta
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input, LSTM,Dense,RepeatVector, TimeDistributed , LeakyReLU
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, IsolationForest
from sklearn.metrics import r2_score, make_scorer, accuracy_score, mean_squared_error, mean_absolute_error
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier, KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
import tensorflow as tf
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
import pandas as pd
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)
import time
from sklearn.pipeline import make_pipeline, Pipeline
import math
import keras
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Input,Conv2D, Conv1D, Conv1DTranspose,MaxPooling1D

import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from keras import optimizers, Sequential
from keras.layers import Attention, GlobalMaxPooling1D
from tensorflow.keras.layers import Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import ConvLSTM1D, Conv1D, BatchNormalization, RepeatVector, TimeDistributed, AveragePooling1D, UpSampling1D
import random
import pickle
import statistics
from tcn import TCN
from sklearn.metrics import f1_score

2023-10-16 13:11:15.550334: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-16 13:11:15.550373: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
import ast
def temporalize(X, lookback=8):

    output_X = []
    #output_y = []
    for i in range(len(X) - lookback - 1):
        t = []
        for j in range(1, lookback + 1):
            # Gather the past records upto the lookback period
            t.append(X[[(i + j + 1)], :])
        output_X.append(t)
        #output_y.append(y[i + lookback + 1])
    #return np.squeeze(np.array(output_X)), np.array(output_y)
    return np.squeeze(np.array(output_X))
def getHyperParams(file, df_hyp):
    param_dict = ast.literal_eval(df_hyp.loc[df_hyp['file'] == file]['Hyperparameters'].values[0])
    return param_dict['model__batch_size'], param_dict['model__encoding_dim'], param_dict['model__epochs'],param_dict['model__layer1'], param_dict['model__layer2'] 

In [4]:
SKAB_RESULTSPATH_TOTAL_TRAIN = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/train/'
SKAB_RESULTSPATH_TOTAL_TEST = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/Iter3/test/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']


In [17]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/LSTM_HYP.csv')

SKAB_RESULTSPATH_TOTAL = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_train =  {}
source_dict_results_test =  {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']    
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = LstmAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df ,train_df= other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_LstmAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_LstmAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = temporalize(train_merge_df)

model = LstmAEModel(layer1=50, layer2=30, encoding_dim=6)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=128, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_train =  {}
mtl_dict_results_test =  {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_LstmAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_LstmAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/100
35/35 - 4s - loss: 0.0752 - val_loss: 0.0628 - 4s/epoch - 127ms/step
Epoch 2/100
35/35 - 1s - loss: 0.0398 - val_loss: 0.0303 - 953ms/epoch - 27ms/step
Epoch 3/100
35/35 - 1s - loss: 0.0211 - val_loss: 0.0186 - 945ms/epoch - 27ms/step
Epoch 4/100
35/35 - 1s - loss: 0.0128 - val_loss: 0.0111 - 949ms/epoch - 27ms/step
Epoch 5/100
35/35 - 1s - loss: 0.0102 - val_loss: 0.0092 - 960ms/epoch - 27ms/step
Epoch 6/100
35/35 - 1s - loss: 0.0090 - val_loss: 0.0089 - 963ms/epoch - 28ms/step
Epoch 7/100
35/35 - 1s - loss: 0.0085 - val_loss: 0.0092 - 959ms/epoch - 27ms/step
Epoch 8/100
35/35 - 1s - loss: 0.0082 - val_loss: 0.0087 - 949ms/epoch - 27ms/step
Epoch 9/100
35/35 - 1s - loss: 0.0079 - val_loss: 0.0083 - 990ms/epoch - 28ms/step
Epoch 10/100
35/35 - 1s - loss: 0.0073 - val_loss: 0.0081 - 960ms/epoch - 27ms/step
Epoch 11/100
35/35 - 1s - loss: 0.0071 - val_loss: 0.0080 - 949ms/epoch - 27ms/step
Epoch 12/100
35/35 - 1s - loss: 0.0068 - val_loss: 0.0080 - 975ms/epoch - 28ms/step
Epo

70/70 - 1s - loss: 0.0041 - val_loss: 0.0055 - 1s/epoch - 21ms/step
Epoch 21/200
70/70 - 2s - loss: 0.0040 - val_loss: 0.0054 - 2s/epoch - 23ms/step
Epoch 22/200
70/70 - 1s - loss: 0.0039 - val_loss: 0.0050 - 1s/epoch - 21ms/step
Epoch 23/200
70/70 - 1s - loss: 0.0039 - val_loss: 0.0052 - 1s/epoch - 21ms/step
Epoch 24/200
70/70 - 1s - loss: 0.0038 - val_loss: 0.0054 - 1s/epoch - 21ms/step
Epoch 25/200
70/70 - 2s - loss: 0.0038 - val_loss: 0.0055 - 2s/epoch - 22ms/step
Epoch 26/200
70/70 - 2s - loss: 0.0038 - val_loss: 0.0054 - 2s/epoch - 21ms/step
Epoch 27/200
70/70 - 1s - loss: 0.0037 - val_loss: 0.0048 - 1s/epoch - 21ms/step
Epoch 28/200
70/70 - 1s - loss: 0.0036 - val_loss: 0.0048 - 1s/epoch - 21ms/step
Epoch 29/200
70/70 - 1s - loss: 0.0035 - val_loss: 0.0047 - 1s/epoch - 21ms/step
Epoch 30/200
70/70 - 1s - loss: 0.0035 - val_loss: 0.0048 - 1s/epoch - 21ms/step
Epoch 31/200
70/70 - 1s - loss: 0.0035 - val_loss: 0.0045 - 1s/epoch - 21ms/step
Epoch 32/200
70/70 - 1s - loss: 0.0035 - 

70/70 - 2s - loss: 0.0032 - val_loss: 0.0045 - 2s/epoch - 25ms/step
Epoch 122/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0047 - 2s/epoch - 25ms/step
Epoch 123/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0046 - 2s/epoch - 25ms/step
Epoch 124/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0047 - 2s/epoch - 25ms/step
Epoch 125/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0047 - 2s/epoch - 25ms/step
Epoch 126/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0046 - 2s/epoch - 25ms/step
Epoch 127/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0046 - 2s/epoch - 25ms/step
Epoch 128/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0047 - 2s/epoch - 25ms/step
Epoch 129/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0045 - 2s/epoch - 22ms/step
Epoch 130/200
70/70 - 1s - loss: 0.0032 - val_loss: 0.0045 - 1s/epoch - 21ms/step
Epoch 131/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0045 - 2s/epoch - 23ms/step
Epoch 132/200
70/70 - 2s - loss: 0.0032 - val_loss: 0.0046 - 2s/epoch - 25ms/step
Epoch 133/200
70/70 - 2s - los

Epoch 22/100
70/70 - 2s - loss: 0.0039 - val_loss: 0.0048 - 2s/epoch - 27ms/step
Epoch 23/100
70/70 - 2s - loss: 0.0039 - val_loss: 0.0049 - 2s/epoch - 28ms/step
Epoch 24/100
70/70 - 2s - loss: 0.0039 - val_loss: 0.0051 - 2s/epoch - 27ms/step
Epoch 25/100
70/70 - 2s - loss: 0.0039 - val_loss: 0.0050 - 2s/epoch - 28ms/step
Epoch 26/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0051 - 2s/epoch - 28ms/step
Epoch 27/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0047 - 2s/epoch - 28ms/step
Epoch 28/100
70/70 - 2s - loss: 0.0039 - val_loss: 0.0047 - 2s/epoch - 27ms/step
Epoch 29/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0047 - 2s/epoch - 27ms/step
Epoch 30/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0048 - 2s/epoch - 28ms/step
Epoch 31/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0047 - 2s/epoch - 27ms/step
Epoch 32/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0049 - 2s/epoch - 27ms/step
Epoch 33/100
70/70 - 2s - loss: 0.0038 - val_loss: 0.0047 - 2s/epoch - 28ms/step
Epoch 34/100
70/70 - 2s - lo

Epoch 24/200
70/70 - 2s - loss: 0.0037 - val_loss: 0.0051 - 2s/epoch - 22ms/step
Epoch 25/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0049 - 2s/epoch - 22ms/step
Epoch 26/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0046 - 2s/epoch - 22ms/step
Epoch 27/200
70/70 - 2s - loss: 0.0037 - val_loss: 0.0049 - 2s/epoch - 24ms/step
Epoch 28/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0047 - 2s/epoch - 22ms/step
Epoch 29/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0051 - 2s/epoch - 22ms/step
Epoch 30/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0049 - 2s/epoch - 22ms/step
Epoch 31/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0048 - 2s/epoch - 22ms/step
Epoch 32/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0052 - 2s/epoch - 22ms/step
Epoch 33/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0050 - 2s/epoch - 22ms/step
Epoch 34/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0048 - 2s/epoch - 22ms/step
Epoch 35/200
70/70 - 2s - loss: 0.0036 - val_loss: 0.0051 - 2s/epoch - 22ms/step
Epoch 36/200
70/70 - 2s - lo

...

Epoch 199/200
70/70 - 1s - loss: 0.0033 - val_loss: 0.0040 - 1s/epoch - 21ms/step
Epoch 200/200
70/70 - 1s - loss: 0.0033 - val_loss: 0.0040 - 1s/epoch - 20ms/step
Epoch 1/200
35/35 - 5s - loss: 0.0738 - val_loss: 0.0848 - 5s/epoch - 144ms/step
Epoch 2/200
35/35 - 1s - loss: 0.0402 - val_loss: 0.0309 - 979ms/epoch - 28ms/step
Epoch 3/200
35/35 - 1s - loss: 0.0187 - val_loss: 0.0106 - 962ms/epoch - 27ms/step
Epoch 4/200
35/35 - 1s - loss: 0.0087 - val_loss: 0.0089 - 957ms/epoch - 27ms/step
Epoch 5/200
35/35 - 1s - loss: 0.0074 - val_loss: 0.0086 - 957ms/epoch - 27ms/step
Epoch 6/200
35/35 - 1s - loss: 0.0070 - val_loss: 0.0087 - 972ms/epoch - 28ms/step
Epoch 7/200
35/35 - 1s - loss: 0.0067 - val_loss: 0.0081 - 976ms/epoch - 28ms/step
Epoch 8/200
35/35 - 1s - loss: 0.0064 - val_loss: 0.0075 - 957ms/epoch - 27ms/step
Epoch 9/200
35/35 - 1s - loss: 0.0063 - val_loss: 0.0079 - 952ms/epoch - 27ms/step
Epoch 10/200
35/35 - 1s - loss: 0.0061 - val_loss: 0.0078 - 973ms/epoch - 28ms/step
Epoch 1

Epoch 97/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0063 - 961ms/epoch - 27ms/step
Epoch 98/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0063 - 976ms/epoch - 28ms/step
Epoch 99/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0064 - 942ms/epoch - 27ms/step
Epoch 100/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0059 - 935ms/epoch - 27ms/step
Epoch 101/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0063 - 954ms/epoch - 27ms/step
Epoch 102/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0062 - 957ms/epoch - 27ms/step
Epoch 103/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0062 - 967ms/epoch - 28ms/step
Epoch 104/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0060 - 972ms/epoch - 28ms/step
Epoch 105/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0066 - 967ms/epoch - 28ms/step
Epoch 106/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0062 - 961ms/epoch - 27ms/step
Epoch 107/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0068 - 947ms/epoch - 27ms/step
Epoch 108/200
35/35 - 1s - loss: 0.0042 - val_loss: 0.0064 - 965ms/e

Epoch 194/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0058 - 939ms/epoch - 27ms/step
Epoch 195/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0060 - 977ms/epoch - 28ms/step
Epoch 196/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0056 - 989ms/epoch - 28ms/step
Epoch 197/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0063 - 984ms/epoch - 28ms/step
Epoch 198/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0057 - 968ms/epoch - 28ms/step
Epoch 199/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0062 - 989ms/epoch - 28ms/step
Epoch 200/200
35/35 - 1s - loss: 0.0040 - val_loss: 0.0060 - 968ms/epoch - 28ms/step
Epoch 1/100
70/70 - 6s - loss: 0.0619 - val_loss: 0.0386 - 6s/epoch - 80ms/step
Epoch 2/100
70/70 - 1s - loss: 0.0205 - val_loss: 0.0107 - 1s/epoch - 21ms/step
Epoch 3/100
70/70 - 1s - loss: 0.0097 - val_loss: 0.0149 - 1s/epoch - 21ms/step
Epoch 4/100
70/70 - 1s - loss: 0.0076 - val_loss: 0.0104 - 1s/epoch - 21ms/step
Epoch 5/100
70/70 - 1s - loss: 0.0063 - val_loss: 0.0070 - 1s/epoch - 21ms/step
Epoch

Epoch 95/100
70/70 - 1s - loss: 0.0036 - val_loss: 0.0042 - 1s/epoch - 21ms/step
Epoch 96/100
70/70 - 1s - loss: 0.0036 - val_loss: 0.0042 - 1s/epoch - 21ms/step
Epoch 97/100
70/70 - 1s - loss: 0.0036 - val_loss: 0.0042 - 1s/epoch - 21ms/step
Epoch 98/100
70/70 - 1s - loss: 0.0036 - val_loss: 0.0043 - 1s/epoch - 21ms/step
Epoch 99/100
70/70 - 1s - loss: 0.0036 - val_loss: 0.0044 - 1s/epoch - 21ms/step
Epoch 100/100
70/70 - 1s - loss: 0.0035 - val_loss: 0.0043 - 1s/epoch - 21ms/step
Epoch 1/100
109/109 - 6s - loss: 0.0472 - val_loss: 0.0624 - 6s/epoch - 58ms/step
Epoch 2/100
109/109 - 3s - loss: 0.0250 - val_loss: 0.0595 - 3s/epoch - 23ms/step
Epoch 3/100
109/109 - 3s - loss: 0.0232 - val_loss: 0.0550 - 3s/epoch - 24ms/step
Epoch 4/100
109/109 - 3s - loss: 0.0211 - val_loss: 0.0485 - 3s/epoch - 23ms/step
Epoch 5/100
109/109 - 3s - loss: 0.0204 - val_loss: 0.0478 - 3s/epoch - 23ms/step
Epoch 6/100
109/109 - 3s - loss: 0.0203 - val_loss: 0.0472 - 3s/epoch - 23ms/step
Epoch 7/100
109/109 -

Epoch 94/100
109/109 - 3s - loss: 0.0123 - val_loss: 0.0365 - 3s/epoch - 24ms/step
Epoch 95/100
109/109 - 3s - loss: 0.0123 - val_loss: 0.0366 - 3s/epoch - 23ms/step
Epoch 96/100
109/109 - 3s - loss: 0.0122 - val_loss: 0.0363 - 3s/epoch - 23ms/step
Epoch 97/100
109/109 - 3s - loss: 0.0123 - val_loss: 0.0366 - 3s/epoch - 24ms/step
Epoch 98/100
109/109 - 3s - loss: 0.0122 - val_loss: 0.0366 - 3s/epoch - 24ms/step
Epoch 99/100
109/109 - 3s - loss: 0.0122 - val_loss: 0.0368 - 3s/epoch - 23ms/step
Epoch 100/100
109/109 - 3s - loss: 0.0122 - val_loss: 0.0366 - 3s/epoch - 23ms/step


## LSTMATTNAE

In [18]:
df_hyp = pd.read_csv('/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/LSTMATTNAE_HYP.csv')

SKAB_RESULTSPATH_TOTAL = '/mnt/stud/home/vpulagura/experiment/SKAB/HyperParamsNew/Results/'
input_features = ['Accelerometer1RMS', 'Accelerometer2RMS', 'Current','Pressure', 'Temperature', 'Thermocouple', 'Voltage','Volume Flow RateRMS']
total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
source_dict_results_train =  {}
source_dict_results_test =  {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    model__batch_size,model__encoding_dim, model__epochs, model__layer1, model__layer2 = getHyperParams(file,df_hyp)                                               
    model  = LstmAttnAEModel(layer1=model__layer1, layer2=model__layer2, encoding_dim=model__encoding_dim)
    model.fit(xtrain, xtrain, epochs=model__epochs, batch_size=model__batch_size, verbose=2,validation_split=0.1, callbacks=callback1)
    test_df, train_df = other_evaluation(xtrain, xtest, ytest, ytrain,X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    source_dict_results_test[file] = test_df
    source_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_STL_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_STL_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(source_dict_results_train, f)
train_merge_df = data_unimodel(anomaly_free_train, source_list, df_dict_total)
random.shuffle(train_merge_df)
train_merge_df = temporalize(train_merge_df)

model = LstmAttnAEModel(layer1=40, layer2=40, encoding_dim=6)
model.fit(train_merge_df, train_merge_df, epochs=100, batch_size=256, verbose=2,validation_split=0.1, callbacks=callback1)

total_list, source_list,df_dict_total,anomaly_free_train = getSKABData()
callback1 = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=10, verbose=1)
lst=[]
lookback = 8
mtl_dict_results_train =  {}
mtl_dict_results_test =  {}
for file in total_list:
    df= df_dict_total[file]
    xtrain, xtest, ytrain, ytest, X_index_test = get_train_test_data(df,anomaly_free_train,input_features)
    xtrain = temporalize(xtrain)
    xtest = temporalize(xtest)
    ytrain, ytest = ytrain[lookback+1:], ytest[lookback+1:]
    ytest = ytest.reset_index()
    ytest = ytest['anomaly']
    test_df, train_df = other_evaluation(xtrain, xtest, ytest,ytrain, X_index_test, model)
    lst.append(f1_score(y_true=test_df['ytrue'], y_pred=test_df['ypred_mse']))
    mtl_dict_results_test[file] = test_df
    mtl_dict_results_train[file] = train_df
with open(SKAB_RESULTSPATH_TOTAL_TEST+'test_MTL_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_test, f)
with open(SKAB_RESULTSPATH_TOTAL_TRAIN+'train_MTL_LstmAttnAE.pkl', 'wb') as f:
    pickle.dump(mtl_dict_results_train, f)

Epoch 1/100
70/70 - 5s - loss: 0.0602 - val_loss: 0.0194 - 5s/epoch - 77ms/step
Epoch 2/100
70/70 - 1s - loss: 0.0137 - val_loss: 0.0102 - 1s/epoch - 17ms/step
Epoch 3/100
70/70 - 1s - loss: 0.0078 - val_loss: 0.0086 - 1s/epoch - 17ms/step
Epoch 4/100
70/70 - 1s - loss: 0.0072 - val_loss: 0.0075 - 1s/epoch - 17ms/step
Epoch 5/100
70/70 - 1s - loss: 0.0072 - val_loss: 0.0072 - 1s/epoch - 17ms/step
Epoch 6/100
70/70 - 1s - loss: 0.0070 - val_loss: 0.0073 - 1s/epoch - 17ms/step
Epoch 7/100
70/70 - 1s - loss: 0.0069 - val_loss: 0.0068 - 1s/epoch - 17ms/step
Epoch 8/100
70/70 - 1s - loss: 0.0069 - val_loss: 0.0065 - 1s/epoch - 17ms/step
Epoch 9/100
70/70 - 1s - loss: 0.0068 - val_loss: 0.0068 - 1s/epoch - 18ms/step
Epoch 10/100
70/70 - 1s - loss: 0.0068 - val_loss: 0.0064 - 1s/epoch - 17ms/step
Epoch 11/100
70/70 - 1s - loss: 0.0067 - val_loss: 0.0066 - 1s/epoch - 17ms/step
Epoch 12/100
70/70 - 1s - loss: 0.0067 - val_loss: 0.0070 - 1s/epoch - 17ms/step
Epoch 13/100
70/70 - 1s - loss: 0.006

Epoch 77/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 939ms/epoch - 27ms/step
Epoch 78/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0046 - 929ms/epoch - 27ms/step
Epoch 79/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0046 - 949ms/epoch - 27ms/step
Epoch 80/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 962ms/epoch - 27ms/step
Epoch 81/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 951ms/epoch - 27ms/step
Epoch 82/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 947ms/epoch - 27ms/step
Epoch 83/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0046 - 973ms/epoch - 28ms/step
Epoch 84/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0046 - 943ms/epoch - 27ms/step
Epoch 85/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 942ms/epoch - 27ms/step
Epoch 86/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0048 - 967ms/epoch - 28ms/step
Epoch 87/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0050 - 967ms/epoch - 28ms/step
Epoch 88/100
35/35 - 1s - loss: 0.0035 - val_loss: 0.0047 - 979ms/epoch - 28

Epoch 78/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0061 - 1s/epoch - 16ms/step
Epoch 79/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0061 - 1s/epoch - 16ms/step
Epoch 80/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0059 - 1s/epoch - 16ms/step
Epoch 81/100
70/70 - 1s - loss: 0.0043 - val_loss: 0.0061 - 1s/epoch - 16ms/step
Epoch 82/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0058 - 1s/epoch - 16ms/step
Epoch 83/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0062 - 1s/epoch - 16ms/step
Epoch 84/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0060 - 1s/epoch - 16ms/step
Epoch 85/100
70/70 - 1s - loss: 0.0044 - val_loss: 0.0061 - 1s/epoch - 16ms/step
Epoch 86/100
70/70 - 1s - loss: 0.0043 - val_loss: 0.0060 - 1s/epoch - 16ms/step
Epoch 87/100
70/70 - 1s - loss: 0.0043 - val_loss: 0.0063 - 1s/epoch - 16ms/step
Epoch 88/100
70/70 - 1s - loss: 0.0043 - val_loss: 0.0061 - 1s/epoch - 16ms/step
Epoch 89/100
70/70 - 1s - loss: 0.0043 - val_loss: 0.0060 - 1s/epoch - 16ms/step
Epoch 90/100
70/70 - 1s - lo

Epoch 77/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0058 - 827ms/epoch - 24ms/step
Epoch 78/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0060 - 816ms/epoch - 23ms/step
Epoch 79/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0061 - 830ms/epoch - 24ms/step
Epoch 80/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0061 - 878ms/epoch - 25ms/step
Epoch 81/100
35/35 - 1s - loss: 0.0040 - val_loss: 0.0058 - 814ms/epoch - 23ms/step
Epoch 82/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0059 - 845ms/epoch - 24ms/step
Epoch 83/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0059 - 824ms/epoch - 24ms/step
Epoch 84/100
35/35 - 1s - loss: 0.0040 - val_loss: 0.0060 - 836ms/epoch - 24ms/step
Epoch 85/100
35/35 - 1s - loss: 0.0040 - val_loss: 0.0062 - 839ms/epoch - 24ms/step
Epoch 86/100
35/35 - 1s - loss: 0.0040 - val_loss: 0.0057 - 822ms/epoch - 23ms/step
Epoch 87/100
35/35 - 1s - loss: 0.0041 - val_loss: 0.0059 - 831ms/epoch - 24ms/step
Epoch 88/100
35/35 - 1s - loss: 0.0040 - val_loss: 0.0054 - 824ms/epoch - 24

Epoch 75/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0048 - 773ms/epoch - 22ms/step
Epoch 76/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0047 - 766ms/epoch - 22ms/step
Epoch 77/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0047 - 772ms/epoch - 22ms/step
Epoch 78/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0049 - 752ms/epoch - 21ms/step
Epoch 79/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0048 - 756ms/epoch - 22ms/step
Epoch 80/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0049 - 773ms/epoch - 22ms/step
Epoch 81/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0048 - 760ms/epoch - 22ms/step
Epoch 82/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0047 - 770ms/epoch - 22ms/step
Epoch 83/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0050 - 773ms/epoch - 22ms/step
Epoch 84/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0048 - 786ms/epoch - 22ms/step
Epoch 85/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0047 - 777ms/epoch - 22ms/step
Epoch 86/100
35/35 - 1s - loss: 0.0038 - val_loss: 0.0047 - 762ms/epoch - 22

...

Epoch 31/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0076 - 823ms/epoch - 24ms/step
Epoch 32/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0073 - 831ms/epoch - 24ms/step
Epoch 33/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0076 - 829ms/epoch - 24ms/step
Epoch 34/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0075 - 844ms/epoch - 24ms/step
Epoch 35/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0072 - 836ms/epoch - 24ms/step
Epoch 36/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0070 - 807ms/epoch - 23ms/step
Epoch 37/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0075 - 829ms/epoch - 24ms/step
Epoch 38/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0085 - 821ms/epoch - 23ms/step
Epoch 39/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0073 - 833ms/epoch - 24ms/step
Epoch 40/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0075 - 814ms/epoch - 23ms/step
Epoch 41/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0078 - 832ms/epoch - 24ms/step
Epoch 42/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0083 - 825ms/epoch - 24

Epoch 29/100
35/35 - 1s - loss: 0.0046 - val_loss: 0.0056 - 884ms/epoch - 25ms/step
Epoch 30/100
35/35 - 1s - loss: 0.0046 - val_loss: 0.0059 - 896ms/epoch - 26ms/step
Epoch 31/100
35/35 - 1s - loss: 0.0046 - val_loss: 0.0054 - 881ms/epoch - 25ms/step
Epoch 32/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0055 - 869ms/epoch - 25ms/step
Epoch 33/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0055 - 888ms/epoch - 25ms/step
Epoch 34/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0052 - 887ms/epoch - 25ms/step
Epoch 35/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0058 - 877ms/epoch - 25ms/step
Epoch 36/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0054 - 896ms/epoch - 26ms/step
Epoch 37/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0054 - 879ms/epoch - 25ms/step
Epoch 38/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0058 - 884ms/epoch - 25ms/step
Epoch 39/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0058 - 903ms/epoch - 26ms/step
Epoch 40/100
35/35 - 1s - loss: 0.0045 - val_loss: 0.0056 - 970ms/epoch - 28

Epoch 27/100
35/35 - 1s - loss: 0.0057 - val_loss: 0.0087 - 1s/epoch - 29ms/step
Epoch 28/100
35/35 - 1s - loss: 0.0057 - val_loss: 0.0086 - 940ms/epoch - 27ms/step
Epoch 29/100
35/35 - 1s - loss: 0.0057 - val_loss: 0.0098 - 942ms/epoch - 27ms/step
Epoch 30/100
35/35 - 1s - loss: 0.0056 - val_loss: 0.0091 - 973ms/epoch - 28ms/step
Epoch 31/100
35/35 - 1s - loss: 0.0056 - val_loss: 0.0080 - 939ms/epoch - 27ms/step
Epoch 32/100
35/35 - 1s - loss: 0.0056 - val_loss: 0.0080 - 930ms/epoch - 27ms/step
Epoch 33/100
35/35 - 1s - loss: 0.0056 - val_loss: 0.0085 - 950ms/epoch - 27ms/step
Epoch 34/100
35/35 - 1s - loss: 0.0056 - val_loss: 0.0095 - 973ms/epoch - 28ms/step
Epoch 35/100
35/35 - 1s - loss: 0.0055 - val_loss: 0.0077 - 954ms/epoch - 27ms/step
Epoch 36/100
35/35 - 1s - loss: 0.0055 - val_loss: 0.0082 - 932ms/epoch - 27ms/step
Epoch 37/100
35/35 - 1s - loss: 0.0055 - val_loss: 0.0080 - 952ms/epoch - 27ms/step
Epoch 38/100
35/35 - 1s - loss: 0.0055 - val_loss: 0.0078 - 946ms/epoch - 27ms/

Epoch 25/100
35/35 - 1s - loss: 0.0061 - val_loss: 0.0081 - 849ms/epoch - 24ms/step
Epoch 26/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0097 - 850ms/epoch - 24ms/step
Epoch 27/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0106 - 836ms/epoch - 24ms/step
Epoch 28/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0080 - 830ms/epoch - 24ms/step
Epoch 29/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0095 - 828ms/epoch - 24ms/step
Epoch 30/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0101 - 828ms/epoch - 24ms/step
Epoch 31/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0096 - 834ms/epoch - 24ms/step
Epoch 32/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0098 - 848ms/epoch - 24ms/step
Epoch 33/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0096 - 839ms/epoch - 24ms/step
Epoch 34/100
35/35 - 1s - loss: 0.0060 - val_loss: 0.0091 - 836ms/epoch - 24ms/step
Epoch 35/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0105 - 863ms/epoch - 25ms/step
Epoch 36/100
35/35 - 1s - loss: 0.0059 - val_loss: 0.0097 - 829ms/epoch - 24

Epoch 24/100
55/55 - 1s - loss: 0.0213 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 25/100
55/55 - 1s - loss: 0.0213 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 26/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 27/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0488 - 1s/epoch - 26ms/step
Epoch 28/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0486 - 1s/epoch - 26ms/step
Epoch 29/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0486 - 1s/epoch - 26ms/step
Epoch 30/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 31/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 32/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0486 - 1s/epoch - 26ms/step
Epoch 33/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0487 - 1s/epoch - 26ms/step
Epoch 34/100
55/55 - 2s - loss: 0.0212 - val_loss: 0.0486 - 2s/epoch - 27ms/step
Epoch 35/100
55/55 - 1s - loss: 0.0212 - val_loss: 0.0486 - 1s/epoch - 26ms/step
Epoch 36/100
55/55 - 1s - lo